#                            DIABETIC RECTINOPATHY DETECTION

---


                         
                         
  Diabetic rectiopathy is an disease of eye caused to an person due to diabetes,it is the condition in which person's eye is damaged due to mellitus present in human eye.
  




> ![](https://afamilyoptician.co.uk/wp-content/uploads/2017/05/diabetic-retinopathy-v01.png)


---



# Diabetic rectiopathy has several stages

---




0 - No DR

1 - Mild

2 - Moderate

3 - Severe

4 - Proliferative DR


---





# **IMPORT LIBRARIES**


---



First of all import the required libraries
I am using fastai library for my project at this time as this an type of image classification problem I am using fatai.vision to solve the problem.

In [ ]:
import numpy as np 
import pandas as pd 
import os
files = os.listdir("../input")
print(files)
print('trainlabels.csv' in files)
print(len(files))

In [ ]:
from fastai import *
from fastai.vision import *
import matplotlib as plt
import pandas as pd
from fastai.widgets import ClassConfusion
from fastai.widgets import *

> Checking if an GPU is enabled or not
Make sure that cuda is installed and is available.

In [ ]:
print('Make sure cuda is installed:', torch.cuda.is_available())
print('Make sure cudnn is enabled:', torch.backends.cudnn.enabled)

# **READ DATA**


---


Read the data which is given in an csv 

In [ ]:
train_df =  pd.read_csv("../input/aptos2019-blindness-detection/train.csv")
valid_df =  pd.read_csv("../input/aptos2019-blindness-detection/test.csv")


Look at the data i.e dataframe of train images and also for the test images.

In [ ]:
print(len(train_df))
train_df.head(10)


In [ ]:
print(len(valid_df))
valid_df.head(10)

> Look through the training data

In [ ]:
train_df['diagnosis'].hist(figsize = (10, 5))


# **TRANSFORMING IMAGES**


---



Transforming the images for bettor model training 


Transform the images of same type
Example all the images are initially not of the same size and shape so we have to transform them in specific order for the better trainig the neural network.
Here it rotate the image either vertical or flip or zoom it according to its need.

In [ ]:
#tfms = get_tranforms(do_flip=True,)
tfms=get_transforms(do_flip = True,flip_vert = True,max_rotate=360,max_zoom = 1.1)

# **CREATING DATABUNCH**


---


1. Now the data is ready to Bunch them together to fit the model for Trainig.
2. Here I use ImageList to bunch them together.
3. Am also splitting them by and random.
4.  labbelling them according to the indexes.
5.  and finnaly normalize them here normalization means arranging the things in specific order 

In [ ]:
data = (ImageList.from_df(train_df,"../input/aptos2019-blindness-detection/train_images",suffix='.png').
       split_by_rand_pct(0.1).
       label_from_df(1).
       transform(tfms,size=256).
       databunch(bs = 20).
       normalize(imagenet_stats))

In [ ]:
data.show_batch(20)

# **CREATING LEANER**


---


making an Architecture
create learn(A object for the neural network) here I am choosing Pretrained Model resnet101 for my problem to solve.

In [ ]:
learn = cnn_learner(data, models.resnet101, metrics=accuracy,model_dir="/kaggle/working")


# **LEARNING RATE ANALYSIS**


---


For finding appropriate learning rate we go through the data once and plot the graph of it.

In [ ]:
learn.lr_find()

plot the graph of learning rate finder.

In [ ]:
learn.recorder.plot()

# **MODLE TRAINING**


---



Here learning rate is slice(2e-5,2e-3) which means first layer will have the learning rate as 2e-5 and last layer(not an output layer) will have learning rate of 2e-3 other all hidden layers will have learning rate of in between 2e-5 to 2e-3

In [ ]:
learn.fit_one_cycle(5,slice(2e-5,2e-3),wd=0.1,moms=(0.8,0.9))

Train a bit more

In [ ]:
learn.fit_one_cycle(2,max_lr=slice(2.5e-3),wd=0.1,moms=(0.8,0.9))

In [ ]:
learn.fit_one_cycle(2,slice(2.5e-5),wd=0.01,moms=(0.8,0.9))

In [ ]:
learn.fit_one_cycle(6,max_lr=slice(1e-3,1e-4),wd=0.1,moms=0.9)

# **SAVE THE MODULE**


---


Now we got enough accurate results so save the Trained weights.

In [ ]:
learn.save('stage-1')

Loading the Trained Model named "stage-1"

In [ ]:
learn.load('stage-1')

Freeze the weights for not furthur modification

In [ ]:
learn.freeze()

# **PLOT THE LOSSES**


---



In [ ]:
learn.recorder.plot_losses()

show results will display the result of any batch in this case first batch.

# **CLASSIFICATION INTERPRETATION**


---



In [ ]:
learn.show_results()

In [ ]:
learn.get_preds()

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)

losses,idxs = interp.top_losses()

len(data.valid_ds)==len(losses)==len(idxs)

# **CONFUSION MATRIX**


---



In [ ]:
interp.plot_confusion_matrix()

In [ ]:
interp.most_confused()

> Transforming the images for prediction

In [ ]:
Tf = partial(Image.apply_tfms,tfms=get_transforms(do_flip=True, flip_vert = True)[0][1:]+get_transforms(do_flip=True, flip_vert = True)[1],size = 512)  

In [ ]:
sub = pd.read_csv("../input/aptos2019-blindness-detection/sample_submission.csv")

In [ ]:
print(sub)

# **SINGLE IMAGE TEST**


---



In [ ]:
img = open_image("../input/aptos2019-blindness-detection/test_images/01c31b10ab99.png")
pre = learn.predict(img)
x = pre[1]
x = int(x)
img.show()
print('SERVITY_LEVEL:',(x))

> Predicting the images and writting the prediction on submission.csv file for submission

# **ALL TEST IMAGE RESULTS**


---



In [ ]:
for i in range(len(sub.id_code)):
    s=0
    id = sub.id_code[i]
    img=open_image("../input/aptos2019-blindness-detection/test_images/"+sub.id_code[i]+".png")
    """
    for i in range(10):
            Img = Tf(img)
            p = learn.predict(Img)
            p = p[1]
            p = int(p)
            #print(p) 
            s=s+p
    """
            
    Img = Tf(img)
    s = learn.predict(Img)
    s = s[1]
    s = int(s)
    print(s)
    sub.diagnosis[i]=s
    print(sub.diagnosis[i])

In [ ]:
 print(sub) 

**END PROCESSING**